#Set up Google Drive



In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Use GPU if Avaliable

In [43]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Task 1

### Encode NER labels

In [44]:
NER_LABELS = {
    'B-ORG': 0,
    'O': 1,
    'B-MISC': 2,
    'B-PER': 3, 
    'I-PER': 4,
    'B-LOC': 5,
    'I-ORG': 6,
    'I-MISC': 7,
    'I-LOC': 8
}

### Preprocessing Train Data

In [45]:
vocab = {}
train_sentences = []
train_sentences_NER = []

NER_lables_counter = {}
Total_NER_labels = 0

with open('/content/drive/MyDrive/Dataset/given/train.txt', 'r') as train_file:
    vocab_unique_val = 1
    sentence_buffer = []
    NER_seq_buffer = []
    
    while True:
        line = train_file.readline()
        # if eof
        if not line:
            # edge case
            indexed_sentence = [vocab[word] for word in sentence_buffer]
            encoded_NER_seq = [NER_LABELS[label] for label in NER_seq_buffer]
            train_sentences.append(indexed_sentence)
            train_sentences_NER.append(encoded_NER_seq)
            break
        # if not blank line
        if line.strip():
            word = line.split(" ")[1].strip()          
            label = line.split(" ")[2].strip()

            # build the vocab
            if word not in vocab:
                vocab[word] = vocab_unique_val
                vocab_unique_val += 1                           

            if label not in NER_lables_counter:
                NER_lables_counter[label] = 1
            else:
                NER_lables_counter[label] = NER_lables_counter[label] + 1

            sentence_buffer.append(word)
            NER_seq_buffer.append(label)

            Total_NER_labels += 1
        # if blank line      
        else:
            indexed_sentence = [vocab[word] for word in sentence_buffer]
            encoded_NER_seq = [NER_LABELS[label] for label in NER_seq_buffer]
            train_sentences.append(indexed_sentence)
            train_sentences_NER.append(encoded_NER_seq)
            sentence_buffer = []
            NER_seq_buffer = []

print(train_sentences)
print(train_sentences_NER)
print(vocab)

class_frequencies = {label: count / Total_NER_labels for label, count in NER_lables_counter.items()}
class_weights = {label: 1 / freq for label, freq in class_frequencies.items()}
print(class_weights)

[[1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11], [12, 13], [14, 15, 16, 17, 18, 19, 20, 21, 22, 3, 23, 5, 24, 5, 25, 7, 8, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 5, 36, 9], [37, 38, 39, 5, 40, 15, 41, 38, 42, 43, 44, 45, 17, 18, 46, 24, 47, 48, 49, 50, 51, 52, 53, 54, 26, 40, 55, 23, 56, 57, 9], [58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 60, 61, 68, 63, 69, 70, 20, 71, 58, 40, 16, 38, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 9], [82, 17, 83, 55, 84, 56, 85, 86, 87, 20, 56, 88, 89, 90, 56, 91, 20, 47, 34, 92, 93, 40, 15, 41, 9], [82, 17, 79, 94, 95, 96, 93, 1, 97, 98, 99, 100, 5, 101, 36, 102, 71, 103, 86, 104, 105, 50, 40, 106, 86, 107, 108, 109, 56, 79, 110, 111, 86, 112, 113, 5, 114, 106, 115, 9], [100, 116, 117, 118, 119, 120, 50, 54, 86, 121, 89, 122, 123, 124, 36, 125, 126, 127, 128, 129, 130, 131, 132, 133, 30, 31, 32, 9], [134, 100, 135, 5, 136, 137, 94, 119, 40, 1, 38, 138, 42, 43, 71, 139, 107, 115, 140, 71, 141, 87, 64, 90, 56, 142, 143, 144, 56, 145, 79, 146, 147, 5, 106, 115, 9], [148

### Preprocessing Dev Data

In [46]:
dev_sentences = []
dev_sentences_NER = []

with open('/content/drive/MyDrive/Dataset/given/dev.txt', 'r') as test_file:
    temp = 0
    test = 0

    sentence_buffer = []
    NER_seq_buffer = []

    while True:
        line = test_file.readline()
        # if eof
        if not line:
            # edge case    
            indexed_sentence = [vocab[word] if word in vocab else len(vocab) + 1 for word in sentence_buffer]    
            encoded_NER_seq = [NER_LABELS[label] for label in NER_seq_buffer]  
            dev_sentences.append(indexed_sentence)   
            dev_sentences_NER.append(encoded_NER_seq)            
            break
        # if not blank line
        if line.strip():
            word = line.split(" ")[1].strip()          
            label = line.split(" ")[2].strip()
            sentence_buffer.append(word)
            NER_seq_buffer.append(label)

        # if blank line      
        else:       
            indexed_sentence = [vocab[word] if word in vocab else len(vocab) + 1 for word in sentence_buffer]  
            encoded_NER_seq = [NER_LABELS[label] for label in NER_seq_buffer]   
            dev_sentences.append(indexed_sentence)
            dev_sentences_NER.append(encoded_NER_seq)
            sentence_buffer = []
            NER_seq_buffer = []

print(dev_sentences)
print(dev_sentences_NER)
print(vocab)

[[2174, 677, 23625, 7303, 2132, 1779, 14592, 2341, 12261, 14042, 9], [270, 21694], [1064, 6046, 2498, 1829, 2226, 812, 2362, 70, 9899, 18, 1163, 143, 2223, 1902, 2195, 93, 157, 2275, 86, 4113, 3282, 236, 407, 1008, 5, 216, 823, 156, 40, 1956, 162, 40, 6955, 2725, 9], [1151, 5337, 18, 398, 71, 6677, 71, 1985, 34, 15945, 143, 2143, 404, 2212, 71, 10359, 86, 2249, 430, 1449, 236, 18, 1940, 497, 2215, 3735, 319, 70, 3117, 382, 236, 201, 18831, 1909, 789, 2243, 9], [1766, 13975, 2195, 833, 70, 6934, 18, 40, 393, 3754, 156, 21015, 13868, 71, 2223, 1267, 201, 1395, 2275, 93, 2293, 3282, 1083, 1713, 6849, 833, 70, 23625, 22, 1434, 23625, 3079, 16466, 2150, 1051, 70, 6934, 9], [14046, 93, 23625, 71, 2195, 3262, 79, 6175, 1693, 5, 201, 519, 2275, 1083, 2226, 7501, 236, 5, 23625, 1153, 833, 70, 18238, 9], [2212, 71, 7988, 71, 2768, 3793, 5, 16212, 201, 398, 1033, 119, 16454, 2283, 86, 10, 11421, 422, 1153, 79, 1191, 15474, 18, 201, 1909, 789, 2259, 156, 2258, 9], [2283, 71, 425, 1643, 5, 1434, 38

### Prepare Train Dataloader

In [47]:
def custom_collate_fn(batch):
    inputs, labels = zip(*batch)
    lengths = torch.tensor([len(sequence) for sequence in inputs])
    padded_inputs = pad_sequence([torch.tensor(sequence) for sequence in inputs], batch_first=True, padding_value=0)
    padded_labels = pad_sequence([torch.tensor(label_sequence) for label_sequence in labels], batch_first=True, padding_value=9)
    return padded_inputs, padded_labels, lengths

In [48]:
from torch.utils.data import DataLoader,TensorDataset
from torch.nn.utils.rnn import pad_sequence

train_data = list(zip(train_sentences, train_sentences_NER))
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, collate_fn=custom_collate_fn)

### Prepare Dev Dataloader

In [49]:
dev_data = list(zip(dev_sentences, dev_sentences_NER))
dev_loader = DataLoader(dev_data, batch_size=256, shuffle=True, collate_fn=custom_collate_fn)

### Dataloader for Prediction

In [50]:
predict_data = list(zip(dev_sentences, dev_sentences_NER))
predict_loader = DataLoader(dev_data, batch_size=256, shuffle=False, collate_fn=custom_collate_fn)

### Bi-directional LSTM Constructor

In [51]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, lstm_hidden_dim, lstm_layers, lstm_dropout, linear_output_dim, num_classes):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 2, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim, num_layers=lstm_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(lstm_dropout)
        self.linear = nn.Linear(lstm_hidden_dim * 2, linear_output_dim)
        self.activation = nn.ELU()
        self.classifier = nn.Linear(linear_output_dim, num_classes)

    def forward(self, inputs, lengths):
        embeddings = self.embedding(inputs)
        packed_embeddings = pack_padded_sequence(embeddings, lengths, batch_first=True, enforce_sorted=False)
        packed_lstm_output, _ = self.lstm(packed_embeddings)
        lstm_output, _ = pad_packed_sequence(packed_lstm_output, batch_first=True)
        lstm_output = self.dropout(lstm_output)
        linear_output = self.activation(self.linear(lstm_output))
        logits = self.classifier(linear_output)
        return logits


### Train Bidirectional LSTM

In [52]:
class_weights_list = list(class_weights.values())
class_weights_tensor = torch.tensor(class_weights_list, dtype=torch.float)


model = BiLSTM(len(vocab), embedding_dim=100, lstm_hidden_dim=256, lstm_layers=1, lstm_dropout=0.33, linear_output_dim=128, num_classes=9)
criterion = nn.CrossEntropyLoss(ignore_index=9, weight=class_weights_tensor)
optimizer = torch.optim.RAdam(model.parameters(), lr=0.001)

#patience = 10
#best_val_loss = float('inf')
#epochs_without_improvement = 0

for epoch in range(100):
    running_loss = 0.0
    print(epoch)
    for i, (inputs, labels, lengths) in enumerate(train_loader):

        lengths = torch.sum(inputs != 0, dim=1)  # Calculate the lengths of input sequences
        optimizer.zero_grad()
        outputs = model(inputs, lengths)
        loss = criterion(outputs.view(-1, 9), labels.view(-1))
        loss.backward()
        optimizer.step()       
        running_loss += loss.item()

        #print('[Epoch %d, Iteration %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))

    model.eval()
    # Validation loop
    correct = 0
    total = 0
    total_loss = 0
    with torch.no_grad():
        for inputs, labels, lengths in dev_loader:
            outputs = model(inputs, lengths)
            _, predicted = torch.max(outputs, dim=2)
            # Count the number of non-padding values in the batch
            non_pad_elements = torch.sum(labels != 9)
            total += non_pad_elements.item()

            # Count the number of correct predictions in the batch
            correct += torch.sum(torch.masked_select(predicted == labels, inputs != 0)).item()

            batch_loss = criterion(outputs.view(-1, 9), labels.view(-1))
            total_loss += batch_loss.item()
    avg_loss = total_loss / len(dev_loader)
    accuracy = 100 * correct / total

    print('Validation Loss: %.16f' % avg_loss)
    print('Accuracy of the model on the %d dev sequences: %.16f %%' % (total, accuracy))
    """
    if avg_loss < best_val_loss:
        best_val_loss = avg_loss
        epochs_without_improvement = 0
        torch.save(model, "/content/drive/MyDrive/Dataset/debug_model.pt")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping")
            break
    """
    model.train()

0
Validation Loss: 2.1401747124535695
Accuracy of the model on the 51578 dev sequences: 31.1314901702276181 %
1
Validation Loss: 1.8343771014894759
Accuracy of the model on the 51578 dev sequences: 33.5317383380511060 %
2
Validation Loss: 1.5054134215627397
Accuracy of the model on the 51578 dev sequences: 48.2647640466865724 %
3
Validation Loss: 1.2815404534339905
Accuracy of the model on the 51578 dev sequences: 52.9818915041296705 %
4
Validation Loss: 1.1409274850572859
Accuracy of the model on the 51578 dev sequences: 54.0404823762069100 %
5
Validation Loss: 1.0203113172735487
Accuracy of the model on the 51578 dev sequences: 56.6617550118267488 %
6
Validation Loss: 0.9275940528937748
Accuracy of the model on the 51578 dev sequences: 64.5178176742021776 %
7
Validation Loss: 0.8829758933612278
Accuracy of the model on the 51578 dev sequences: 67.3213385551979542 %
8
Validation Loss: 0.8816750177315303
Accuracy of the model on the 51578 dev sequences: 70.6095622164488788 %
9
Validati

# Model Cache

In [53]:
torch.save(model, "/content/drive/MyDrive/Dataset/debug_model.pt")
model = torch.load("/content/drive/MyDrive/Dataset/debug_model.pt")
model.eval()

BiLSTM(
  (embedding): Embedding(23626, 100, padding_idx=0)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (linear): Linear(in_features=512, out_features=128, bias=True)
  (activation): ELU(alpha=1.0)
  (classifier): Linear(in_features=128, out_features=9, bias=True)
)

# Check if Prediction is reasonable


In [54]:
input_sequences = []
predictions = []

with torch.no_grad():
    for inputs, labels, lengths in predict_loader:
        outputs = model(inputs, lengths)
        _, predicted = torch.max(outputs, dim=2)
        
        # Remove padding from inputs and predictions
        for seq_idx, seq_length in enumerate(lengths):
            input_sequences.append(inputs[seq_idx, :seq_length].tolist())
            predictions.append(predicted[seq_idx, :seq_length].tolist())

for i in range(100):
    print(predictions[i])


[1, 1, 3, 4, 1, 1, 1, 1, 1, 1, 1]
[5, 1]
[2, 7, 1, 5, 8, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 8, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 5, 1]
[3, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 5, 1, 0, 1, 3, 4, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
[1, 1, 1, 1, 1, 5, 1, 3, 4, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

# Create Evaluation Script

In [55]:
input_file = '/content/drive/MyDrive/Dataset/given/dev.txt'
output_file = '/content/drive/MyDrive/Dataset/predict_dev.txt'
REVERSE_NER_LABELS = {v: k for k, v in NER_LABELS.items()}

with open(input_file, 'r') as in_file, open(output_file, 'w') as out_file:
    sentence_index = 0
    word_index = 0
    
    while True:
        line = in_file.readline()
        # if eof
        if not line:
            # edge case            
            break
        # if not blank line
        if line.strip(): 
           label_num =  predictions[sentence_index][word_index]                
           out_file.write(line.strip() + " " + REVERSE_NER_LABELS[label_num] + "\n")  
           #print(repr(line.strip() + " " + REVERSE_NER_LABELS[label_num] + "\n"))
           word_index += 1
         
        # if blank line      
        else:
            out_file.write("\n")
            sentence_index += 1
            word_index = 0